In [1]:
from google.colab import drive
drive.mount('/content/drive')


ModuleNotFoundError: No module named 'google.colab'

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/final

/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/final


In [ ]:
%cd /content/drive/MyDrive/Colab\ Notebooks/DLBasics2023_colab/final/dl_lecture_competition_pub/

/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/final/dl_lecture_competition_pub


In [ ]:
!pip install -r requirements.txt

In [ ]:
import os
import sys
import math
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchmetrics import Accuracy
from termcolor import cprint
from tqdm import tqdm
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

sys.path.append('/content/drive/MyDrive/Colab Notebooks/DLBasics2023_colab/final/dl_lecture_competition_pub/')
from src.datasets import ThingsMEGDataset
from src.utils import set_seed

class Args:
    seed = 42
    batch_size = 10
    num_workers = 0
    data_dir = 'data'
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    lr = 0.0001
    epochs = 200
    patience = 20

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :, :]
        return self.dropout(x)

class TransformerClassifier(nn.Module):
    def __init__(self, num_classes, seq_len, num_channels, d_model=128, nhead=8, num_layers=4, dropout=0.2):
        super().__init__()
        self.normalize = nn.BatchNorm1d(num_channels)
        self.embedding = nn.Linear(num_channels, d_model)
        self.bn1 = nn.BatchNorm1d(d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout, max_len=seq_len)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, dim_feedforward=512, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.layer_norm = nn.LayerNorm(d_model)
        self.fc = nn.Linear(d_model, num_classes)
        self.seq_len = seq_len

    def forward(self, x):
        if x.shape[2] != self.seq_len:
            raise ValueError(f"Expected input sequence length {self.seq_len}, but got {x.shape[2]}")

        x = self.normalize(x)
        x = self.embedding(x.transpose(1, 2))
        x = self.bn1(x.permute(0, 2, 1)).permute(0, 2, 1)
        x = x.permute(1, 0, 2)
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x)
        x = self.layer_norm(x)
        x = x.mean(dim=0)
        return self.fc(x)

def mixup_data(x, y, alpha=0.2):
    lam = np.random.beta(alpha, alpha)
    batch_size = x.size()[0]
    index = torch.randperm(batch_size).to(x.device)
    mixed_x = lam * x + (1 - lam) * x[index]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

def mixup_criterion(criterion, pred, y_a, y_b, lam):
    return lam * criterion(pred, y_a) + (1 - lam) * criterion(pred, y_b)

def add_noise(x, noise_level=0.01):
    return x + noise_level * torch.randn_like(x)

def time_shift(x, max_shift=5):
    shift = torch.randint(-max_shift, max_shift + 1, (x.shape[0],))
    return torch.stack([torch.roll(x[i], shifts=shift[i].item(), dims=1) for i in range(x.shape[0])])

args = Args()
set_seed(args.seed)

loader_args = {"batch_size": args.batch_size, "num_workers": args.num_workers}

train_set = ThingsMEGDataset("train", args.data_dir)
train_loader = torch.utils.data.DataLoader(train_set, shuffle=True, **loader_args)
val_set = ThingsMEGDataset("val", args.data_dir)
val_loader = torch.utils.data.DataLoader(val_set, shuffle=False, **loader_args)
test_set = ThingsMEGDataset("test", args.data_dir)
test_loader = torch.utils.data.DataLoader(test_set, shuffle=False, **loader_args)

model = TransformerClassifier(
    train_set.num_classes,
    seq_len=train_set.seq_len,
    num_channels=train_set.num_channels
).to(args.device)

optimizer = torch.optim.AdamW(model.parameters(), lr=args.lr, weight_decay=1e-5)
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=2)

accuracy = Accuracy(
    task="multiclass", num_classes=train_set.num_classes, top_k=10
).to(args.device)

max_val_acc = 0
best_epoch = 0
logdir = 'outputs'

for epoch in range(args.epochs):
    print(f"Epoch {epoch+1}/{args.epochs}")

    model.train()
    train_loss, train_acc = [], []
    for X, y, subject_idxs in tqdm(train_loader, desc="Train"):
        X, y = X.to(args.device), y.to(args.device)
        X = add_noise(X)
        X = time_shift(X)
        X, y_a, y_b, lam = mixup_data(X, y)

        y_pred = model(X)
        loss = mixup_criterion(F.cross_entropy, y_pred, y_a, y_b, lam)
        train_loss.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        optimizer.step()

        acc = accuracy(y_pred, y)
        train_acc.append(acc.item())

    scheduler.step()

    model.eval()
    val_loss, val_acc = [], []
    with torch.no_grad():
        for X, y, subject_idxs in tqdm(val_loader, desc="Validation"):
            X, y = X.to(args.device), y.to(args.device)
            y_pred = model(X)
            val_loss.append(F.cross_entropy(y_pred, y).item())
            val_acc.append(accuracy(y_pred, y).item())

    print(f"Epoch {epoch+1}/{args.epochs} | train loss: {np.mean(train_loss):.3f} | train acc: {np.mean(train_acc):.3f} | val loss: {np.mean(val_loss):.3f} | val acc: {np.mean(val_acc):.3f}")
    torch.save(model.state_dict(), os.path.join(logdir, "model_last.pt"))

    if np.mean(val_acc) > max_val_acc:
        cprint("New best.", "cyan")
        torch.save(model.state_dict(), os.path.join(logdir, "model_best.pt"))
        max_val_acc = np.mean(val_acc)
        best_epoch = epoch

    if epoch - best_epoch >= args.patience:
        cprint("Early stopping", "red")
        break

model.load_state_dict(torch.load(os.path.join(logdir, "model_best.pt"), map_location=args.device))

preds = []
model.eval()
with torch.no_grad():
    for X, subject_idxs in tqdm(test_loader, desc="Test"):
        preds.append(model(X.to(args.device)).cpu())

preds = torch.cat(preds, dim=0).numpy()
np.save(os.path.join(logdir, "submission"), preds)
cprint(f"Submission {preds.shape} saved at {logdir}", "cyan")

Epoch 1/200


Validation: 100%|██████████| 1644/1644 [00:10<00:00, 151.59it/s]


Epoch 1/200 | train loss: 7.547 | train acc: 0.008 | val loss: 7.510 | val acc: 0.009
New best.
Epoch 2/200


Validation: 100%|██████████| 1644/1644 [00:10<00:00, 152.58it/s]


Epoch 2/200 | train loss: 7.508 | train acc: 0.010 | val loss: 7.495 | val acc: 0.013
New best.
Epoch 3/200


...

Epoch 47/200 | train loss: 5.637 | train acc: 0.178 | val loss: 8.019 | val acc: 0.037
Epoch 48/200


Validation: 100%|██████████| 1644/1644 [00:10<00:00, 155.84it/s]


Epoch 48/200 | train loss: 5.566 | train acc: 0.189 | val loss: 8.056 | val acc: 0.039
Early stopping


Test: 100%|██████████| 1644/1644 [00:08<00:00, 184.54it/s]


Submission (16432, 1854) saved at outputs
